<img src="unicamp.png" width="150" height="150">

## MO444/MC886 - Aprendizado de Máquina e Reconhecimento de Padrões

Esse trabalho foi feito pelos seguintes membros:

- Lucas Zanco Ladeira - 188951
- Rafael - 

O código original deste projeto está disponível em [repository inside Github](https://github.com/lucaslzl/p1_clustering). 

## Modelos de Agrupamento e Redução de Dimensionalidade

## I - Introdução

Neste trabalho foi necessário implementar dois modelos de agrupamento e utilizar o algoritmo <b>Principal Component Analysis (PCA)</b> da biblioteca scikit-learn para a tarefa de redução de dimensionalidade. Os modelos implementados compreendem o <b>KMeans</b> e o <b>DBScan</b>. De forma resumida, no primeiro caso são selecionados centróides de clusters e analisados os clusters formados por esses centróides. O algoritmo é iterado um determinado número de vezes e clusters são encontrados na qual a distância entre os membros dos clusters e os centróides seja mínima dentro das iterações. No segundo caso, é calculada a distância entre todos os registros para encontrar quais formam clusters considerando uma distância máxima e uma vizinhança mínima. Os registros são classificados como <i>outlier</i>, borda, e central. Sendo assim, é um algoritmo interessante para identificar <i>outliers</i> dentro de conjuntos de dados.

## II - Implementação

Nesta seção será descrito o código fonte dos algoritmos implementados. Para tal, será sub-dividia em: a) KMeans, b) DBScan, c) PCA. Mesmo sendo que foi utilizada uma biblioteca para implementar o PCA, o código fonte será disponibilizado neste relatório.

### II - a) KMeans

### II - b) DBScan

Esse algoritmo possui dois métodos principais, seguindo o padrão utilizado pela biblioteca scikit-learn, esses compreendem <i>fit</i> e <i>predict</i>. O primeiro tem o intuito de treinar o modelo, ou seja, identificar o comportamento dos registros. Os registros são iterados e classificados de acordo com <i>outlier</i>, borda ou central. Além disso, caso seja um registro central é atribuído a um novo cluster. As variáveis nc (node classification) e ci (cluster id) armazenam essas informações. A seguir, é descrito este método com comentários na língua inglesa para facilitar a extensão a partir da disponibilização do código implementado.

In [ ]:
def fit(self, x):

    # Initialize with 0's
    nc = [0] * len(x)
    ci = [0] * len(x)

    cluster_id = 0

    # Iterate through all records
    for i in tqdm(range(len(x))):

        # If already classified, skip
        if nc[i] != 0:
            continue

        # Get neighbors
        neighbors = self._get_neighbors(x, i)

        # Verify if it is an outlier
        if len(neighbors) < self.min_neighbors:
            nc[i] = -1
            continue

        cluster_id += 1

        # Core record
        nc[i] = 2
        ci[i] = cluster_id

        # Iterate through each neighbor
        indx = 0
        while True:

            # If list of neighbors ended
            if indx == len(neighbors):
                break

            j = neighbors[indx]

            if i == j:
                indx += 1
                continue

            # At least it is a border point
            nc[j] = 1
            ci[j] = cluster_id

            post_neighbors = self._get_neighbors(x, j)

            # Verify if neighbor is core point
            if len(post_neighbors) >= self.min_neighbors:
                # Classify as core point
                nc[j] = 2

                # Continue exploring neighbourhood
                neighbors.extend(post_neighbors)
                neighbors = list(set(neighbors))

            indx += 1

    return (nc, ci)

É possível observar algumas chamadas para o método <i>_get_neighbors<i>. Este método tem o intuito de buscar todos os vizinhos de um determinado registro. Um método chamado <i>_verify_neighbor</i> faz o cálculo da distância euclidiana entre dois registros e retorna <i>True</i> se for vizinho, e <i>False</i> se não for vizinho.

In [ ]:
def _verify_neighbor(self, point_a, point_b):

    # Calculate euclidean distance
    calc_dist = distance.euclidean(point_a, point_b)

    # Append distance to verify description
    self.summed_dist.append(calc_dist)

    # Verify if it is a neighbor
    if calc_dist <= self.distance:
        return True, self.distance

    return False, self.distance


def _get_neighbors(self, x, i):
    
    # Neighbor list
    neighbors = []

    for j in range(len(x)):

        if i == j:
            continue

        # Verify if it is a neighbor
        verif, _ = self._verify_neighbor(x[i], x[j])
        
        if verif:
            # Append to the list of neighbors
            neighbors.append(j)

    return neighbors

Agora será descrito o método <i>predict</i> que faz a predição dos clusters para novos registros considerando os registros centrais já identificados. O método faz a identificação de qual é o registro central mais próximo.

In [ ]:
def predict(self, x, res, y):
    
    (nc, ci) = res
    
    # Initialize with 0's
    ci_pred = [0] * len(y)

    for i in tqdm(range(len(y))):

        # Get neighbors
        neighbors = self._get_neighbors_predict(x, i, y)

        if len(neighbors) > 0:

            # Get closest neighbors
            neighbors = self._get_by_closest(neighbors)

            for j in range(len(neighbors)):
                
                # Get closest neighbors
                indx_j = int(neighbors[j][0])

                # Verify if core point
                if nc[indx_j] == 2:
                    ci_pred[i] = ci[indx_j]
                    break

    return ci_pred

Este método utiliza um método distinto para obter os vizinhos, pois é necessário considerar cada vizinho, como também, as distâncias para os vizinhos. O método tem nome <i>_get_neighbors_predict</i>. Um outro método, chamado <i>_get_by_closest</i>, é utilizado para ordenar todos os vizinhos de acordo com a distância calculada.

In [ ]:
def _get_neighbors_predict(self, x, i, y):

    # Neighbor list
    neighbors = []

    for j in range(len(x)):

        # Verify if it is a neighbor
        verif, dist = self._verify_neighbor(y[i], x[j])

        if verif:
            neighbors.append((j, dist))

    return neighbors


def _get_by_closest(self, neighbors):

    neighbors = np.array(neighbors)
    return neighbors[neighbors[:, 1].argsort()]

### II - c) Principal Componente Analysis

O algoritmo do PCA foi implementado utilizando a biblioteca [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). De acordo com a [wikipedia](https://en.wikipedia.org/wiki/Principal_component_analysis) o "<i>Principal component analysis (PCA) is the process of computing the principal components and using them to perform a change of basis on the data, sometimes using only the first few principal components and ignoring the rest</i>". Sendo assim, caso seja necessário reduzir um conjunto de dados para 3 dimensões é necessário apenas obter os 3 principais componentes. Para facilitar a utilização, foi criada uma classe nova chamada <i>OurPCA</i> com o método <i>fit_transform</i>, o qual cria um objeto PCA e faz a transformações nos dados.

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

class OurPCA:

    def fit_transform(self, data, n_components):

        pca = PCA(n_components=n_components, random_state=42)
        return pca.fit_transform(data)

Além das classes e métodos descritos, outros algoritmos foram implementados para gerenciar os dados, resultados, e experimentos. Estes compreendem:
- <i>main.py</i><br>
Une tudo o que foi implementado para executar os experimentos.<br>


- <i>inout.py</i><br>
Encapsula métodos de leitura de arquivos, persistência de resultados, transformações nos dados, e criação de gráficos.

## III - Metodologia de Avaliação

### III - a) Bases de dados

Neste trabalho são utilizadas duas bases de dados. A primeira foi disponibilizada pela professora Esther, e possui 2 <i>features</i> numéricas. Para descrever os registros da base de dados o método <i>describe</i> da biblioteca pandas é utilizada.

In [1]:
import pandas as pd

df = pd.read_csv('datasets/cluster.dat', sep=' ')

df.describe()

,x,y
count,573.000000,573.000000
mean,1849.808028,15.227836
std,900.129972,8.292268
min,335.000000,1.950000
25%,1155.000000,7.450000
50%,1655.000000,17.200000
75%,2350.000000,22.750000
max,3635.000000,29.150000


A segunda base de dados se refere a registros de históricos de cartões de crédito. O intuito da tarefa de agrupamento é identificar perfis de usuários para campanhas de marketing. Essa base de dados foi obtida do [Kaggle](https://www.kaggle.com/arjunbhasin2013/ccdata). Ela possui 18 <i>features</i> numéricas com alguns valores nulos e 8950 registros. Os valores nulos são preenchidos com 0's.

In [3]:
df = pd.read_csv('datasets/credit.csv')

df.describe()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
count,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8.950000e+03,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8949.000000,8.950000e+03,8.637000e+03,8950.000000,8950.000000
mean,1620.986304,7.714003,1003.204834,592.437371,411.067645,3.270640e+03,4.452865,2.030237,4.117664,2.214069,3.248827,14.709832,4494.449450,5.404793e+03,1.809551e+03,3.809273,11.517318
std,4385.370311,73.859272,2136.634782,1659.887917,904.338115,1.311675e+05,52.604114,29.744300,54.021898,32.210553,6.824647,24.857649,3638.815725,1.702333e+05,3.856941e+04,47.047820,1.338331
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000e+00,1.916300e-02,0.000000,6.000000
25%,128.281915,0.900000,39.635000,0.000000,0.000000,0.000000e+00,0.083333,0.000000,0.000000,0.000000,0.000000,1.000000,1600.000000,3.833158e+02,1.692297e+02,0.000000,12.000000
50%,874.387676,1.000000,361.280000,38.000000,89.000000,0.000000e+00,0.500000,0.083333,0.166667,0.000000,0.000000,7.000000,3000.000000,8.577677e+02,3.128075e+02,0.000000,12.000000
75%,2056.395445,1.000000,1110.130000,577.405000,468.637500,1.114220e+03,1.000000,0.333333,0.750000,0.250000,4.000000,17.000000,6500.000000,1.906756e+03,8.260137e+02,0.166667,12.000000
max,311357.000000,875.000000,49039.570000,40761.250000,22500.000000,1.024992e+07,875.000000,875.000000,875.000000,1125.000000,123.000000,358.000000,30000.000000,1.089444e+07,2.559345e+06,875.000000,12.000000


### III - b) Experimentos

É possível separar os experimentos em 2 grupos principais:
- Modelos de agrupamento

Nesse grupo são apresentados os resultados obtidos e é discutido o impacto de variar os hiperparâmetros dos modelos.


- Redução de Dimensionalidade

## IV - Resultados

### IV - a) Modelos de Agrupamento 

<b>DBScan - Distância Máxima</b>

Para avaliar o DBScan é executado o treinamento do modelo em ambos os datasets variando a distância máxima e o número mínimo de vizinhos. Para encontrar um valor de distância que permita encontrar vizinhos, o algoritmo é executado e todas as distâncias calculadas são armazenadas. Com isso é obtida a média, mediana e moda das distâncias. Esse processo é executado para ambas as bases de dados sem/com a transformação de um <i>Scaler</i>. O <i>Scaler</i> utilizado faz a divisão de cada registro pelo valor máximo encontrado para cada <i>feature</i> da base de dados.

Os diferentes valores resultantes encontrados permitem analisar a necessidade do <i>Scaler</i> para manter uma relação da distância entre cada <i>feature</i> distinta. No caso da utilização da primeira base de dados sem o <i>Scaler</i> foi possível obter os seguintes resultados apresentados a seguir. Encontramos que para a primeira base de dados, a segunda <i>feature</i> possui valor máximo de 29,15, mesmo assim, média da distância entre os pontos é de 1008,851 e mediana 845,234. Ou seja, a distância é afetada pela diferença de escala de ambas as <i>features</i>. Também é possível observar que a primeira <i>feature</i> chega ao valor de 3635,00.
- Média: 1008,851 
- Mediana: 845,234
- Moda: 20,143

Ao aplicar um <i>Scaler</i> em ambas as <i>features</i> da primeira base de dados encontramos os resultados apresentados a seguir. Como o <i>Scaler</i> muda a escala dos dados fazendo que ambos variem entre 0 e 1, existe uma consistência maior na diferença de distância. Os resultados encontrados para a média das distância é de 0,467, ou seja, é próximo do valor médio do intervalo da escala calculada.
- Média: 0,467
- Mediana: 0,530
- Moda: 0,027

Considerando a segunda base de dados sem o uso do <i>Scaler</i> foram encontrados os resultados apresentados a seguir. Nessa base de dados existem <i>features</i> com valor máximo de 2,55 até <i>features</i> com valor máximo de 311357,00. É possível observar que essas diferenças em escala impactam diretamente o algoritmo durante o uso da distância máxima tendo que a média resultante foi de 22313,913. Além disso, como essa base de dados possui uma quantidade maior de <i>features</i>, a diferença em escala impacta um maior número de variáveis.
- Média: 22313,913
- Mediana: 5427,695
- Moda: 1019,243

Ao aplicar um <i>Scaler</i> em todas as <i>features</i> foram encontrados os resultados apresentados a seguir. Nessa base de dados é possível observar que a média das distância foi de 0,259, ou seja, a maior parte das distâncias estão distribuídas no intervalo de 0 até 0,5. Observando os valores de cada <i>feature</i> que foram apresentados anteriormente junto com essa média, temos que o não uso do <i>Scaler</i> implica que a minoria das <i>features</i> iria impactar no resultado final do modelo.
- Média: 0,259
- Mediana: 0,192
- Moda: 0,034

Essas diferenças na escala dos dados e que resultam na distância, fazem com que o algoritmo ignore <i>features</i> com escalas menores ou que dê menos peso para essas <i>features</i>. Mesmo sendo que descrevemos os efeitos encontrados no DBScan, como o KMeans também é baseado na distância ele sofre dos mesmos problemas.

<b>DBScan - Variação Hiperparâmetros</b>

## V - Conclusões

## VI - Apêndice

<span style="color: red">Links, figuras, etc</span>

### Links

- Scikit-learn (https://scikit-learn.org/stable/)